In [3]:
path_wrf_out = f'/home/sagus/Development/temp/wrfout_A_d01_2020-02-06_18:00:00'

In [55]:
from netCDF4 import Dataset
from osgeo import gdal_array, gdal, osr
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import cartopy.crs as crs
import xarray as xr
from cartopy.feature import NaturalEarthFeature

from wrf import (to_np, getvar, smooth2d, get_cartopy, cartopy_xlim,
                 cartopy_ylim, latlon_coords)

In [11]:
wrfout = Dataset(path_wrf_out)

In [9]:
import wrf

In [36]:
WSPD10 = wrf.getvar(wrfout, 'wspd10', timeidx=wrf.ALL_TIMES)

In [31]:
WDIR10 = wrf.getvar(wrfout, 'wdir10', timeidx=wrf.ALL_TIMES)

In [33]:
T2 = wrf.getvar(wrfout, 'T2', timeidx=wrf.ALL_TIMES)

In [85]:
T2

<xarray.DataArray 'T2' (Time: 58, south_north: 269, west_east: 269)>
array([[[293.81305, 293.4371 , 293.04083, ..., 295.52032, 295.58313,
         295.55872],
        [293.20084, 292.84338, 292.48666, ..., 295.74042, 295.69867,
         295.60883],
        [292.45514, 292.125  , 291.82144, ..., 296.02524, 295.84448,
         295.66495],
        ...,
        [287.55817, 287.86786, 288.58176, ..., 308.2441 , 308.23123,
         308.24722],
        [287.2085 , 287.52707, 288.2849 , ..., 308.36002, 308.36517,
         308.40125],
        [286.92038, 287.2457 , 288.04575, ..., 308.42926, 308.45093,
         308.50375]],

       [[295.36728, 294.6246 , 294.1783 , ..., 295.30975, 295.25043,
         295.2559 ],
        [294.0895 , 293.6297 , 293.37054, ..., 295.38397, 295.3806 ,
         295.3916 ],
        [292.3308 , 292.12024, 292.4211 , ..., 295.44247, 295.42282,
         295.32788],
...
        [280.70575, 279.66214, 279.83362, ..., 301.5493 , 301.615  ,
         301.6414 ],
        [280.43112, 278.03403, 278.25165, ..., 301.63797, 301.6298 ,
         301.78552],
        [280.81937, 278.0263 , 278.3224 , ..., 301.6807 , 301.69446,
         301.78018]],

       [[293.564  , 293.35895, 293.31024, ..., 291.56497, 291.58713,
         291.60406],
        [292.12347, 292.08682, 292.2735 , ..., 291.5425 , 291.5811 ,
         291.66736],
        [291.13544, 291.1473 , 291.62622, ..., 291.5651 , 291.61844,
         291.72327],
        ...,
        [280.41162, 279.35324, 279.52267, ..., 300.72906, 300.75003,
         300.77692],
        [280.13013, 277.45245, 277.92422, ..., 300.7544 , 300.75998,
         300.96747],
        [280.4953 , 277.74976, 278.0412 , ..., 300.8112 , 300.8475 ,
         300.99286]]], dtype=float32)
Coordinates:
    XLONG    (south_north, west_east) float32 -68.91031 -68.864136 ... -57.32242
    XLAT     (south_north, west_east) float32 -37.21706 ... -27.518177
    XTIME    (Time) float32 0.0 60.0 120.0 180.0 ... 3240.0 3300.0 3360.0 3420.0
  * Time     (Time) datetime64[ns] 2020-02-06T18:00:00 ... 2020-02-09T03:00:00
Dimensions without coordinates: south_north, west_east
Attributes:
    FieldType:    104
    MemoryOrder:  XY 
    description:  TEMP at 2 M
    units:        K
    stagger:      
    coordinates:  XLONG XLAT XTIME
    projection:   LambertConformal(stand_lon=-62.70000076293945, moad_cen_lat...

In [115]:
for t in range(int(T2.coords['Time'])):
    print(t)

TypeError: only size-1 arrays can be converted to Python scalars

In [62]:
raw = gdal_array.OpenArray(T2[0].values)

In [152]:
wrfout.dimensions['Time'].size

58

In [127]:
for date in T2.coords['Time'].values:
    print(date.d)

IndexError: invalid index to scalar variable.

In [155]:
str(T2.coords['Time'].values[56])[:16]

'2020-02-09T02:00'

In [151]:
str(a)[:16]

'2020-02-06T18:00'